In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn import metrics

In [ ]:
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score, all_nrmse

In [87]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [57]:
len(train_df)

39607

In [ ]:
train_df

In [ ]:
train_x = train_df.filter(regex='X')
train_y = train_df.filter(regex='Y')

In [ ]:
train_x.describe()

In [ ]:
train_y.describe()

In [ ]:
len(train_df)

In [ ]:
y_spec = pd.read_csv('./meta/y_feature_spec_info.csv')
y_spec.head()

In [ ]:
spec = {}
for _, row in y_spec.iterrows():
    spec[row.Feature] = (row.최소, row.최대)
spec

In [ ]:
less_remove_idx = set([])
more_remove_idx = set([])

for idx, row in train_df.iterrows():
    for col, val in spec.items():
        minval, maxval = val
        if minval <= row[col] <= maxval:
            continue
            
        if minval > row[col]:
            less_remove_idx.add(idx)
        else:

remove_idx = list(remove_idx)

In [ ]:
len(remove_idx)

In [70]:
sorted(remove_idx[:20])

[0,
 6,
 9,
 12,
 14,
 23,
 33,
 35,
 8216,
 8217,
 16393,
 16398,
 16401,
 16405,
 16406,
 16418,
 24578,
 24593,
 24607,
 32787]

In [58]:
preprocessing_train_df = train_df.drop(train_df.index[remove_idx])
len(preprocessing_train_df)

35690

In [ ]:
preprocessing_train_df.to_csv('preprocessing_train.csv', index=False)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
preprocessing_train_df.describe()

In [ ]:
train_df.loc[remove_idx].describe()

In [60]:
len(df), len(train_df)

(35690, 39607)

In [68]:
train_df

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14,Abnormal
0,TRAIN_00001,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,0.0,0.0,4.34,0.18,13.34,13.37,13.40,13.52,13.44,3.11,3.17,3.06,3.13,1,2.05,2.07,2.05,2.06,2.06,2.35,1.49,1.69,1.46,1.74,12.99,12.88,12.89,12.99,-16.41,-16.36,-17.03,21.20,20.99,21.28,21.09,0.29,1463,1,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256,2.056,1.456,1.680,10.502,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304,0
1,TRAIN_00002,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,0.0,0.0,4.38,0.18,13.33,13.33,13.46,13.51,13.42,2.97,3.11,2.91,3.20,1,2.10,2.10,2.10,2.10,2.17,2.28,1.49,1.67,1.45,1.66,12.92,12.87,12.89,12.93,-16.06,-16.11,-16.74,21.16,21.03,21.16,21.13,0.13,1463,1,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308,1.446,1.184,1.268,18.507,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438,0
2,TRAIN_00003,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,0.0,0.0,4.36,0.15,13.36,13.36,13.39,13.51,13.43,3.04,3.04,3.01,3.12,1,2.07,2.06,2.04,2.07,2.10,2.26,1.49,1.69,1.46,1.68,12.97,12.87,12.87,13.00,-16.16,-16.17,-16.76,21.13,21.03,21.17,21.12,0.14,1468,1,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232,1.251,0.665,0.782,14.082,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370,0
3,TRAIN_00004,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,0.0,0.0,4.33,0.21,13.30,13.33,13.39,13.51,13.40,3.05,3.01,3.02,3.08,1,2.06,2.05,2.03,2.06,2.07,2.12,1.47,1.68,1.47,1.68,12.91,12.97,12.99,12.92,-16.05,-16.03,-16.67,21.18,20.98,21.20,21.09,0.22,1469,1,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225,1.464,1.079,1.052,16.975,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345,0
4,TRAIN_00005,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,0.0,0.0,4.35,0.16,13.35,13.34,13.41,13.50,13.42,3.04,3.07,3.00,3.12,1,2.09,2.06,2.09,2.12,2.33,2.13,1.49,1.68,1.47,1.82,12.96,12.85,12.91,12.96,-16.25,-16.23,-16.85,21.16,20.96,21.18,21.10,0.22,1469,1,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762,0.983,0.646,0.689,15.047,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,TRAIN_39603,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,0.0,0.0,4.36,0.15,13.37,13.38,13.51,13.52,13.46,3.20,3.03,3.06,3.13,1,2.10,2.11,2.08,2.11,2.10,2.20,1.37,1.60,1.36,1.67,12.89,13.00,12.94,12.90,-16.09,-16.15,-16.79,21.17,21.08,21.19,21.19,0.11,1469,1,1,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812,1.382,1.215,1.263,10.874,29.194,16.582,3.410,-26.486,-26.581,-22.772,24.261,-26.491,-26.584,-26.580,0
39603,TRAIN_39604,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,0.0,0.0,4.40,0.13,13.36,13.36,13.47,13.49,13.44,3.15,3.06,3.05,3.06,1,2.12,2.12,2.09,2.12,2.11,2.20,1.40,1.68,1.37,1.77,12.88,12.88,12.89,12.86,-15.70,-15.74,-16.40,21.16,21.09,21.21,21.19,0.12,1458,1,1,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413,1.482,0.606,1.083,8.759,29.859,15.659,3.406,-27.308,-27.203,-24.674,23.427,-27.250,-27.334,-27.325,0
39604,TRAIN_39605,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,0.0,0.0,4.38,0.14,13.38,1

In [ ]:
df = pd.read_csv('./train.csv')

train_df, valid_df = train_test_split(df, train_size=0.8)

train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

valid_x = valid_df.filter(regex='X') # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature

test_x = pd.read_csv('./test.csv').drop(columns=['ID'])

LR = MultiOutputRegressor(LinearRegression()).fit(train_x, train_y)

valid_preds = LR.predict(valid_x)
lg_nrmse(valid_y.values, valid_preds)

In [83]:
df = pd.read_csv('./preprocessing_train.csv')

train_df, valid_df = train_test_split(df, train_size=0.8)

train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

valid_x = valid_df.filter(regex='X') # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature

test_x = pd.read_csv('./test.csv').drop(columns=['ID'])

LR = MultiOutputRegressor(LinearRegression()).fit(train_x, train_y)

valid_preds = LR.predict(valid_x)
lg_nrmse(valid_y.values, valid_preds)

(1.7155244360603166,
 [0.22778038139067436,
  0.32467426674493055,
  0.3136998315106647,
  0.17724481750978086,
  0.0761363753938185,
  0.04309330276730514,
  0.1136721978328232,
  0.02295400235490118,
  0.02279857273321772,
  0.033828919266419226,
  0.03115386295561913,
  0.022846992384340438,
  0.022884513810954424,
  0.02290536430388754])

In [ ]:
df.filter(regex='Y').describe()

In [ ]:
pd.DataFrame(valid_preds).describe()

In [ ]:
test_x = pd.read_csv('./test.csv').drop(columns=['ID'])
preds = LR.predict(test_x)
submit = pd.read_csv('./sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
submit.to_csv('./range_preprocessing.csv', index=False)

In [ ]:
train_df.loc[remove_idx, 'Abnormal'] = 1
train_df['Abnormal'] = 0
train_df.loc[remove_idx]['Abnormal'] = 1

In [82]:
train_df.to_csv('autoencoder_train.csv')

In [84]:
len(remove_idx)

3917

In [85]:
len(df)

35690

In [88]:
len(train_df)

39607

In [90]:
train_df.loc[remove_idx].to_csv('abnormal_train.csv', index=False)